In [20]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from datetime import date
import itertools

In [21]:
data = pd.read_csv("Exercise4_data.csv", parse_dates = True, index_col = 0).dropna()

In [35]:
data.head(14)

,unemployment,prices,central bank,borrowing,fed,interest,inflation,yields,growth,gdp,...,ebay,craigslist,cheap,luxury,affordable,college,Close,inf_change,Close_l1,inf_change_l1
date,,,,,,,,,,,,,,,,,,,,,
2005-08-14,16.430164,41.288136,5.581386,57.428035,4.830301,2.144768,7.016189,43.755624,-1.569231e+01,7.220464,...,-11.311629,0.000000e+00,-1.880510e+00,-1.126294,-10.187067,3.286610,3.067160,0.526133,2.541027,-0.328261
2005-08-21,17.799344,50.463277,-50.232476,-57.428035,-9.660601,2.144768,-10.524283,5.779045,-7.105427e-15,-15.753740,...,33.934886,1.686113e+00,4.701276e+00,6.757764,32.169686,9.859831,3.646934,0.579774,3.067160,0.526133
2005-08-28,17.114754,37.847458,0.000000,0.000000,-0.690043,-1.429845,0.877024,0.000000,-1.647692e+01,-5.251247,...,-26.393800,3.372226e+00,-4.701276e+00,-14.641822,-40.212108,-9.859831,3.646934,0.579774,3.067160,0.526133
2005-09-04,13.691803,95.192090,0.000000,0.000000,-3.450215,4.289536,8.185554,0.000000,1.412308e+01,7.876870,...,3.770543,-1.686113e+00,1.598434e+01,-6.194617,15.548682,0.000000,3.646934,0.579774,3.067160,0.526133
2005-09-11,22.933770,61.932203,55.813862,69.734043,11.040687,-1.787306,-3.215753,0.000000,3.923077e+00,10.502493,...,-11.311629,-2.220446e-16,-9.402552e+00,3.378882,9.650906,14.789747,3.646934,0.579774,3.067160,0.526133
2005-09-18,19.168525,42.435028,-55.813862,-16.408010,-17.251073,-3.574613,-1.461706,24.767334,3.138462e+00,-3.938435,...,18.852714,-1.686113e+00,-5.641531e+00,6.194617,-2.680807,0.000000,4.741834,1.094899,3.646934,0.579774
2005-09-25,16.772459,114.689266,57.674324,-24.612015,18.631159,0.714923,4.092777,-24.767334,-1.255385e+01,-11.815305,...,-15.082172,3.372226e+00,1.421085e-14,-21.399586,11.259390,-11.503136,4.741834,1.094899,3.646934,0.579774
2005-10-02,18.826230,38.994350,-33.488317,123.060075,-4.830301,-7.149226,-6.431507,-72.650847,5.492308e+00,1.969217,...,-7.541086,-3.372226e+00,-5.641531e+00,23.089027,0.536161,8.216526,4.741834,1.094899,3.646934,0.579774
2005-10-09,18.141639,34.406780,13.023235,-114.856070,8.970558,8.936532,4.385118,72.650847,1.098462e+01,-13.128117,...,41.475972,1.686113e+00,6.581786e+00,-1.689441,-16.084843,1.643305,4.741834,1.094899,3.646934,0.579774


## Train your model on the first 850 rows of data.

What is the MSE of your forecast on the rest of the data? The Random Walk? Fill in the Code below to get the answer 

In [23]:
# There are two essential libraries when we are working with lasso
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model

In [24]:
drops = ['inf_change', 'Close']
X = data.drop(drops, axis = 1)
Y = data[['inf_change']]

In [25]:
#lets the data into training and testing. The first 850 will be trainig.
X_train = X.iloc[:850]
Y_train = Y[:850]
X_test = X[850:]
Y_test = Y[850:]

In [31]:
# The predictors need to be scaled before running lasso
scaler = StandardScaler()

# You fit a scaler so that you can apply that same scaling 
# when you get out of sample observations later
scaler.fit(X_train)
X_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [27]:
# Alpha is the penalty applies for higher values of beta
# An alpha of 0 would be equivalent to OLS, a higher alpha will tend to drive more beta values to 0

# Start by creating a lasso object
clf = linear_model.Lasso(alpha=.01)

# Fit the data on the scaled predictors (left) and y (right)
forecaster = clf.fit(X_scaled, Y_train)

## What Variables were included in the model?

In [28]:
# here is the list of columns not -> 0
X.columns[forecaster.coef_!= 0]

Index(['unemployment', 'prices', 'yields', 'rise', 'increases', 'demand',
       'loan', 'raise', 'tickets', 'gold', 'what is inflation',
       'why so expensive', 'gas prices', 'cheap', 'luxury', 'Close_l1',
       'inf_change_l1'],
      dtype='object')

## What is the MSE of your forecast on the rest of the data? The Random Walk? 

In [32]:
# Finish the MSE calculation for each
((Y_test.values.flatten() - forecaster.predict(X_test_scaled))**2).mean()

0.23374852658546336

In [42]:
#Now lets compare the y error with a random walk
((data["inf_change"] - data["inf_change_l1"])**2).mean()

0.2596144744566492